In [1]:
from dataclasses import dataclass
from typing import Any, List

# Utils

In [2]:
def get_members(cls):
    return set([att for att in dir(cls) if not att.startswith("_")])

def get_values(cls):
    temp = [getattr(cls, att) for att in dir(cls) if not att.startswith("_")]
    vals = set()
    for item in temp:
        if type(item) in (int,float,str):
            vals.add(item)
        elif type(item) in (set,List,tuple):
            for i in item:
                vals.add(i)
    return vals

def add_meta_data(cls):
    cls.Members = get_members(cls)
    cls.Values  = get_values(cls)
                
def to_tuples(items):
    tuples = set()
    for item in items:
        tpl = tuple(StringUtils.clean(item).split(" "))
        tuples.add(tpl)
    return tuples
    
class Singleton(object):
    __instance = None
    def __new__(cls, *args):
        if cls.__instance is None:
            cls.__instance = object.__new__(cls, *args)
        return cls.__instance

class StringUtils(object):
    
    @staticmethod
    def init_caps(s):
        return s[0].upper() + s[1:]
    
    @staticmethod
    def clean(s):
        if not s:
            return ""
        return str(s).replace("-"," ").strip().lower()

# Enum

In [4]:
class CompassEnum(Singleton):
    North = "north"
    South = "south"
    East  = "east"
    West  = "west"
    
class VerticalEnum(Singleton):
    Above = "above"
    Below = "below"
        
class SpecialCommandsEnum(object):
    Describe = to_tuples({"describe"})
    Quit     = to_tuples({"quit","exit"})
    Help     = to_tuples({"help","h","?"})
    
class VerbEnum(Singleton):
    Move   = to_tuples({"move", "go"})
    PickUp = to_tuples({"get", "pick up"})
    Up     = to_tuples({"up", "go up", "climb", "climb up"})
    Down   = to_tuples({"down", "go down", "climb down"})
    
add_meta_data(CompassEnum)
add_meta_data(SpecialCommandsEnum)
add_meta_data(VerticalEnum)
add_meta_data(VerbEnum)

# Entities

In [5]:
@dataclass
class Item(object):
    name: str = ""
        
    def __repr__(self):
        return self.name

@dataclass
class Location(object):
    desc: str = "A room"
    name: str = ""
    
    north: Any = None
    south: Any = None
    east:  Any = None
    west:  Any = None

    above: Any = None
    below: Any = None
        
    def add_north(self, locn):
        self.north = locn
        locn.south = self
        
    def add_south(self, locn):
        self.south = locn
        locn.north = self
        
    def add_east(self, locn):
        self.east = locn
        locn.west = self
        
    def add_west(self, locn):
        self.west = locn
        locn.east = self
        
    def __post_init__(self):
        self.desc = self.desc.strip()
        self.name = self.name.strip()
        
        self.items: List[Any]     = []       
            
    def get_locations(self):
        locations: List[Any] = []
        if self.north:
            locations.append(CompassEnum.North)
        if self.south:
            locations.append(CompassEnum.South)
        if self.west:
            locations.append(CompassEnum.West)
        if self.east:
            locations.append(CompassEnum.East)
        if self.above:
            locations.append(VerticalEnum.Above)
        if self.below:
            locations.append(VerticalEnum.Below)
        return locations
        
    def describe(self):
        out(self.desc)
        s = ""
        for name in self.get_locations():
            locn = getattr(self,name)
            if name in CompassEnum.Values:
                s += f"{StringUtils.init_caps(name)} is a {locn.name}. "
            elif name in CompassEnum.Values:
                s += f"To the {name} is a {locn.name}. "
            else:
                raise Exception(f"Unknown location name: {name}")
        if s:
            out("")
            out(s)        
        out("")
        if len(self.items) > 0:
            if len(self.items) == 1:
                str_items = f"a {self.items[0]}"
            elif len(self.items) > 1:                
                str_items = ", ".join([f"a {i}" for i in self.items[:-1]])
                str_items += f" and a {self.items[-1]}"
            out(f"In the {self.name} you find {str_items}.")

# Parser

In [9]:
class Parser(Singleton):
    StopWords = set("a,an,the".split(","))
    
    @classmethod
    def parse(s):
        tokens = [t for t in StringUtils.clean(s).split(" ") 
                  if t not in Parser.StopWords]
        
        if not tokens:
            return False
        
        if len(tokens) == 1:
            action = tokens[0]
            if action in SpecialCommandsEnum.Describe:
                pass
            elif action in SpecialCommandsEnum.Help:
                out("Help requested")
            elif action == SpecialCommandsEnum.Quit:
                out("Quiting...")
                return False
        else:
            pass
        return True

# Game State

In [ ]:
@dataclass
class Player(Singleton):    
    hp: int         = 100
    is_alive: bool  = True
        
input_list = []
def pop_from_list(s):
    global input_list
    if input_list:
        item = input_list[0]
        input_list = input_list[1:]
        print(item)
        return item
    # print("Input exhausted")
    return None
        
out = lambda s: print(s)
# inpt = lambda s: input(s)
inpt = pop_from_list

class Game(Singleton):
            
    def __init__(self):
        self.hp = 100
        self.player = Player()
        self.loop_num = 0
        self.location = None
        self.inventory = set()
        self.visited = set()
        
    def run(self, location):
        while self.player.is_alive and self.loop_num < 1000:            
            self.loop_num += 1
            self.loop()
        out("Game Over")

    def loop(self):
        action = inpt("What do you want to do?")
        if not action:
            self.player.is_alive = False
            return
        

# World Generation

In [7]:
def generate_world():
    intro = Location(
        desc="Dazed, you awaken to find yourself in a large, dank cavern. ",
        name="large cavern"
    )
    locn_west = Location(desc="""
You enter a small cave with a low ceiling. Inside there is a dank smell, and a low, rumbling noise coming from one corner of the room.
Wary, you glance over a see a snout poking out from the side of a pile of rocks.
""",
                        name="dragon room")
    
    locn_east = Location(desc="",
                        name="treasure room")
    
    intro.add_west(locn_west)
    intro.add_east(locn_east)
    
    intro.items.append(Item(name="torch"))
    intro.items.append(Item(name="sword"))
    return intro

intro = generate_world()
intro.describe()

Dazed, you awaken to find yourself in a large, dank cavern.

West is a dragon room. East is a treasure room. 

In the large cavern you find a torch and a sword.


In [8]:
input_list = [
    "move west",
    "move east",
    "quit",
]

game = Game()
game.run(intro)

move west
move east
quit
Game Over
